In [12]:
import torch
import cv2
import pyautogui as pg
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [13]:
path=os.getcwd()+'\\ultralytics_yolov5_master'

In [14]:
model = torch.hub.load(path, 'yolov5s',source='local',force_reload=True)

YOLOv5  2022-11-5 Python-3.7.5 torch-1.13.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [15]:
w=pg.size().width
h=pg.size().height

In [16]:
cap=cv2.VideoCapture(0)

In [17]:
df=pd.DataFrame(columns=['Object_Counts','Time'])
now = datetime.now()
T=str(now).split(' ')[1]

In [18]:
while True:
    ret,frame=cap.read()
# uncomment below portion and comment above line, if you want to do object detction from the screen itself(half portion)
#     frame=np.array(pg.screenshot(region=(0,0,w/2,h)))#take screen recording of half portion of the screen(other half can be used for detecting objects)
#     frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame=cv2.resize(frame,(1200,800))
    res=model(frame)
    res.render()
    for i in range(len(res.xyxy)):
        x=res.pandas().xyxy[i]['name'].value_counts()
        j=str(x)
        c=0
        for m in j.split('\n')[:-1]:
            cv2.putText(frame,str(m),(50, 50+c),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            c+=30
        if cv2.waitKey(1) == ord('w'):# press w to write the objects counts in a frame to csv
            df=df.append({'Object_Counts':str(' , '.join(j.split('\n')[:-1])),'Time':T},ignore_index=True)
            df.to_csv('results.csv')
    if cap:
        frame=frame
    else:
        frame=cv2.resize(frame,(int(w/2),int(h)))
    cv2.imshow('live',frame)
    if cv2.waitKey(1) == ord('q'):# press q to quit
        cv2.destroyAllWindows()
        break

In [11]:
cv2.destroyAllWindows()